# Progress Bar

In [1]:
# for console output
# from tqdm import tqdm

# for jupyter notebook
from tqdm.notebook import tqdm

import pandas as pd
from pandarallel import pandarallel

import time

In [2]:
j = 0
for i in tqdm(range(100)):
    j += i
    time.sleep(1)
    
print(j)

  0%|          | 0/100 [00:00<?, ?it/s]

4950


If you get error on displaying the progress bar:

```bash
pip uninstall ipywidgets
pip install ipywidgets
```

# Dataframe

About dataset:
    https://www.kaggle.com/datasets/lakshmi25npathi/bike-sharing-dataset

In [3]:
df = pd.read_csv("../data/bike.csv")
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [4]:
df.shape

(10886, 12)

## Feature Engineering

### Iterrows()

In [6]:
df["season_name"] = ""
df["weather_name"] = ""

seasons = {1: "spring", 2: "summer", 3: "fall", 4: "winter"}
weather = {1: "clear", 2: "mist", 3: "light", 4: "heavy"}

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    df.loc[index, "season_name"] = seasons[row["season"]]
    df.loc[index, "weather_name"] = weather[row["weather"]]
    

  0%|          | 0/10886 [00:00<?, ?it/s]

In [33]:
df.sample(5)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,season_name,weather_name
2059,2011-05-12 17:00:00,2,0,1,2,26.24,31.060,57,12.9980,54,540,594,summer,mist
7160,2012-04-16 17:00:00,2,1,0,1,32.80,34.850,33,26.0027,111,601,712,summer,clear
6082,2012-02-09 16:00:00,1,0,1,1,13.94,16.665,36,0.0000,15,223,238,spring,clear
1362,2011-04-02 15:00:00,2,0,0,3,13.94,15.150,76,26.0027,72,64,136,summer,light
9020,2012-08-18 05:00:00,3,0,0,1,24.60,28.790,78,16.9979,2,7,9,fall,clear


### Pandas Apply()

In [8]:
tqdm.pandas()

In [14]:
df["season_name"] = ""
df["weather_name"] = ""

def apply_season(x):
    seasons = {1: "spring", 2: "summer", 3: "fall", 4: "winter"}
    time.sleep(0.001)
    return seasons[x]

def apply_weather(x):
    weather = {1: "clear", 2: "mist", 3: "light", 4: "heavy"}
    time.sleep(0.001)
    return weather[x]

df["season_name"] = df["season"].progress_apply(apply_season)
df["weather_name"] = df["weather"].progress_apply(apply_weather)

  0%|          | 0/10886 [00:00<?, ?it/s]

  0%|          | 0/10886 [00:00<?, ?it/s]

In [12]:
df.sample(5)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,season_name,weather_name
2951,2011-07-11 21:00:00,3,0,1,1,28.70,33.335,79,27.9993,21,109,130,fall,clear
2801,2011-07-05 15:00:00,3,0,1,1,33.62,37.120,43,0.0000,51,111,162,fall,clear
7004,2012-04-10 04:00:00,2,0,1,1,13.94,16.665,71,8.9981,0,2,2,summer,clear
9476,2012-09-18 05:00:00,3,0,1,2,26.24,28.790,89,22.0028,3,45,48,fall,mist
4206,2011-10-07 07:00:00,4,0,1,1,17.22,21.210,94,0.0000,5,237,242,winter,clear


## Parallel Apply

In [15]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [16]:
df["season_name"] = ""

def apply_season(x):
    import time
    seasons = {1: "spring", 2: "summer", 3: "fall", 4: "winter"}
    time.sleep(0.001)
    return seasons[x]

t0 = time.time()
df["season_name"] = df["season"].parallel_apply(apply_season)
t1 = time.time()

print(f"It took: {t1-t0:.5f} sec")

It took: 7.64936 sec
